In [ ]:
import json

# TRY THIS OUT
# https://github.com/nlptown/nlp-notebooks/blob/master/Simple%20Sentence%20Similarity.ipynb

import requests
import re

In [ ]:
# link = "https://www.collegevine.com/schools/hub/overall/sheet"

link = "https://www.collegevine.com/schools/hub/data/static"

# avoid printing f.text; Large file
f = requests.get(link)

university_list = re.findall("\"rankings\":.+?\"bucket\":\"(\d*)\",\"key\":\"overall\".+?\"slug\":\"(.+?)\"", f.text)

In [ ]:
university_list = sorted(university_list, key = lambda pair: int(pair[0]))

gap = 0

for x in range(len(university_list)):
    university_list[x] = [int(university_list[x][0]), university_list[x][1]]
    if x+1 != int(university_list[x][0]) - gap:
        gap += 1
        print(university_list[x-1:x+1])

# print(len(sorted(university_list, key = lambda pair: pair[0])))

selected_schools = university_list

In [ ]:
# get links for all schools

college_links = []

for school in selected_schools:
    name = school[1]
    college_links.append("https://www.collegevine.com/schools/" 
                            + name.replace(" ", "-") 
                            +  "/essay-prompts")

len(college_links)

In [ ]:
# access links and create general word bag

# currently trying to use list instead of dictionary to store prompts and their corresponding index
allprompts = []
# prompt_matrix = []
# prompt_corpus = []
# numPrompts = []

potential_issues = []
index = 0
prompts = []
has_common_app = False
word_list = []

# for iteration in range(0, len(college_links), 20):
#     # time.sleep(3)
#     print(iteration)
#     for link_index in range(iteration, iteration + 20):
        # print(link_index)

for link_index in range(len(college_links)):
    has_common_app = False
    link = college_links[link_index]
    f = requests.get(link)

    noprompts = re.search("<h2>", f.text)

    if not bool(noprompts):
        prompts = re.findall("<h3 class='mt-2'>([\S\s]+?)</h3>[\S\s]+?<p>([\S\s]+?)</p>", f.text)
        select_prompts = re.findall("</h5>\s<p>\s<p>([\S\s]+?)</p>", f.text)

        if prompts == []:
            potential_issues.append(link)
            continue

        if prompts[-1][0] == "Common App Personal Essay":
            has_common_app = True
            prompts = prompts[:-1]
            select_prompts = select_prompts[:-7]

        if select_prompts != []:
            i = 0
            while i < len(prompts):
                if "Select-A-Prompt" in prompts[i][0]:
                    del prompts[i]
                else:
                    i += 1

            if has_common_app:
                for x in select_prompts:
                    prompts.append(("Select-A-Prompt", x))

        # numPrompts.append(len(prompts))

        # checks again, but now for schools with only common app essay
        if prompts == []:
            continue
        
        # prompts is a list of prompts for the current school.
        # each prompt is in a tuple with their title first and then the prompt
        allprompts.append([link_index, prompts])
        

        # prompts, word_list = get_bag(prompts, word_list)

        # for x in prompts:
        #     prompt_corpus.append(" ".join(x))
        
        
        # for j in range(len(prompts)):
        #     prompt_matrix.append([index, j] + prompts[j])
        
        # index += 1

In [ ]:
with open('json_data.json', 'w') as outfile:
    json.dump(allprompts, outfile)